# Table of Contents
 <p><div class="lev1"><a href="#How-to-obtain-weather-data-from-MERRA-2-(Part-2):-Download-raw-data"><span class="toc-item-num">1&nbsp;&nbsp;</span>How to obtain weather data from MERRA-2 (Part 2): Download raw data</a></div><div class="lev2"><a href="#About-this-Notebook"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>About this Notebook</a></div><div class="lev3"><a href="#Other-notebooks"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Other notebooks</a></div><div class="lev3"><a href="#License"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>License</a></div><div class="lev3"><a href="#Table-of-contents"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Table of contents</a></div><div class="lev1"><a href="#Script-Setup"><span class="toc-item-num">2&nbsp;&nbsp;</span>Script Setup</a></div><div class="lev1"><a href="#Download-raw-data"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download raw data</a></div><div class="lev2"><a href="#Input"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Input</a></div><div class="lev3"><a href="#Parameters-choices"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Parameters choices</a></div><div class="lev3"><a href="#Timeframe"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Timeframe</a></div><div class="lev3"><a href="#Geography-coordinates"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Geography coordinates</a></div><div class="lev2"><a href="#Subsetting-data"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Subsetting data</a></div><div class="lev2"><a href="#Downloading-data"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Downloading data</a></div><div class="lev1"><a href="#Setting-up-dataframe(s)"><span class="toc-item-num">4&nbsp;&nbsp;</span>Setting up dataframe(s)</a></div><div class="lev2"><a href="#Concatenating/combining-individual-files"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Concatenating/combining individual files</a></div><div class="lev2"><a href="#First-look-at-the-final-data-frame-structure-and-format"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>First look at the final data frame structure and format</a></div><div class="lev2"><a href="#Saving-dataframe"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Saving dataframe</a></div>

# How to obtain weather data from MERRA-2 (Part 2): Download raw data

## About this Notebook
This Jupyter Notebook is part of the [Open Power System Data Project](http://www.open-power-system-data.org) and is written in Python 3. 

This is **Part 2** of the notebook. It aims to download data from the MERRRA-2 weather dataset.

---

### Other notebooks
**Part 1**: Introduction

**Part 3**: Processing raw data and compiling the data package

### License

This notebook is published under [The MIT License](https://opensource.org/licenses/mit-license.php) license:

Copyright (c) 2016 [copyright holders]

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### Table of contents
 <p><div class="lev2"><a href="#About-this-Notebook"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>About this Notebook</a></div><div class="lev3"><a href="#Other-notebooks"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Other notebooks</a></div><div class="lev3"><a href="#License"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>License</a></div><div class="lev3"><a href="#Table-of-contents"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Table of contents</a></div><div class="lev1"><a href="#Script-Setup"><span class="toc-item-num">2&nbsp;&nbsp;</span>Script Setup</a></div><div class="lev1"><a href="#Download-raw-data"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download raw data</a></div><div class="lev2"><a href="#Input"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Input</a></div><div class="lev3"><a href="#Parameters-choices"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Parameters choices</a></div><div class="lev3"><a href="#Timeframe"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Timeframe</a></div><div class="lev3"><a href="#Geography-coordinates"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Geography coordinates</a></div><div class="lev2"><a href="#Subsetting-data"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Subsetting data</a></div><div class="lev2"><a href="#Downloading-data"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Downloading data</a></div><div class="lev1"><a href="#Setting-up-dataframe(s)"><span class="toc-item-num">4&nbsp;&nbsp;</span>Setting up dataframe(s)</a></div><div class="lev2"><a href="#Concatenating/combining-individual-files"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Concatenating/combining individual files</a></div><div class="lev2"><a href="#First-look-at-the-final-data-frame-structure-and-format"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>First look at the final data frame structure and format</a></div><div class="lev2"><a href="#Saving-dataframe"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Saving dataframe</a></div>
 
***

# Script Setup

In [108]:
# importing all necessary Python libraries for this Script
import pandas as pd
import xarray as xr
import numpy as np
import requests
import logging
import os
from datetime import datetime
from calendar import monthrange
from opendap_download.multi_processing_download import DownloadManager
import math
from functools import partial
import re
import getpass
from datetime import datetime, timedelta
import pytz
import dateutil.parser

# Set up a log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger('notebook')


---
# Download raw data

## Input
### Parameters choices
Definition of Input parameters for creating URL with OPeNDAP. Which parameters shall be included in the weather data package?

**general parameters**
- tavg1_2d_slv_Nx = M2T1NXSLV:
  - H850: Height at 850 hPa
  - H500: Height at 500 hPa
  - H250: Height at 250 hPa
  - DISPH: Displacement height
  - ftp://goldsmr4.sci.gsfc.nasa.gov/data/s4pa/MERRA2/M2T1NXSLV.5.12.4/
  - via OPenDAP: http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/contents.html
  
**Wind Speed** 
- tavg1_2d_slv_Nx = M2T1NXSLV
  - U2M: Eastward wind at 2 m above displacement height
  - U10M: Eastward wind at 10 m above displacement height
  - U50M: Eastward wind at 50 m above surface
  - U850: Eastward wind at 850 hPa
  - U500: Eastward wind at 500 hPa
  - U250: Eastward wind at 250 hPa
  - V2M: Northward wind at 2 m above displacement height
  - V10M: Northward wind at 10 m above displacement height
  - V50M: Northward wind at 50 m above surface
  - V850: Northward wind at 850 hPa
  - V500: Northward wind at 500 hPa
  - V250: Northward wind at 250 hPa
  - ftp://goldsmr4.sci.gsfc.nasa.gov/data/s4pa/MERRA2/M2T1NXSLV.5.12.4/
  - via OPenDAP: http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/contents.html
  
- tavg1_2d_flx_Nx = M2T1NXFLX:
  - Z0M: Roughness length, momentum
  - ftp://goldsmr4.sci.gsfc.nasa.gov/data/s4pa/MERRA2/M2T1NXFLX.5.12.4/
  - via OPenDAP: http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/contents.html

später: **Temperature** (tavg1_2d_slv_Nx = M2T1NXSLV)
- TS: Surface skin temperature
- T2M Temperature at 2 m above the displacement height
- T10M: Temperature at 10 m above the displacement height
- T850: Temperature at 850 hPa
- T500: Temperature at 500 hPa
- T250: Temperature at 250 hPa
- ftp://goldsmr4.sci.gsfc.nasa.gov/data/s4pa/MERRA2/M2T1NXSLV.5.12.4/
- via OPenDAP: http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/contents.html

später: **Solar Radiation** (tavg1_2d_rad_Nx = M2T1NXRAD)
- SWGDN: Surface incident shortwave flux (incident = einfallender Strahl)
- SWGDNCLR: Surface incident shortwave flux assuming clear sky
- SWGNT: Surface net downward shortwave flux
- SWGNTCLR: Surface net downward shortwave flux assuming clear sky
- SWGNTCLN: Surface net downward shortwave flux assuming clean sky
- SWGNTCLRCLN: Surface net downward shortwave flux assuming clear clean sky
- SWTDN: top-of-atmosphere incoming shortwave flux
- _possibly more_
- ftp://goldsmr4.sci.gsfc.nasa.gov/data/s4pa/MERRA2/M2T1NXRAD.5.12.4/
- via OPenDAP: http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXRAD.5.12.4/contents.html

### Parameter selection 
These are the possible parameters:
* wind
* solar radiation
* temperature

If you want to select more than one parameter, separate them with commas. For example: wind, solar radiation, temperature

In [43]:
# Getting user input
possible_params = ['wind', 'solar radiation', 'temperature']

### Timeframe

Definition of desired timespan data is needed for. (Optional: daily, monthly, yearly aggregation)

In [74]:
# TODO: User input 
# User input of timespan
download_year = 2014 # Testcase Schlewswig Holstein, 2014, hourly data
# download_month = '01'
# download_day = '01'

download_start_date = str(download_year) + '-01-01'

### Geography coordinates
Definition of desired coordinates (rectangular area) data is needed for -> corner coordinates input

In [45]:
# User input of coordinates
# ------
# Bsp. Schleswig-Holstein (lat/lon in WGS84 Dezimalgrad)
# Nordost-Punkt; 55,036823°N, 11,349297°E
# Südwest-Punkt; 53,366266°N, 7,887088°E

lat_1, lon_1 = 53.366266, 7.887088 
lat_2, lon_2 = 55.036823, 11.349297


def translate_lat_to_geos5_native(latitude):
    """
    The source for this formula is in the MERRA2 
    Variable Details - File specifications for GEOS pdf file.
    
    latitude: float Needs +/- instead of N/S
    """
    return 1 + ((latitude + 90) / 0.5)

def translate_lon_to_geos5_native(longitude):
    """See function above"""
    return (1 + ((longitude) + 180) / 0.625)

def find_closest_coordinate(calc_coord, coord_array):
    index = (np.abs(coord_array-calc_coord)).argmin()
    return coord_array[index]

# The arrays contain the coordinates of the grid used by the API
lat_coords = np.arange(0, 360, dtype=int)
lon_coords = np.arange(0, 575, dtype=int)

lat_coord_1 = translate_lat_to_geos5_native(lat_1)
lon_coord_1 = translate_lon_to_geos5_native(lon_1)
lat_coord_2 = translate_lat_to_geos5_native(lat_2)
lon_coord_2 = translate_lon_to_geos5_native(lon_2)


# find the closest coordinate in the grid
lat_co_1_closest = find_closest_coordinate(lat_coord_1, lat_coords)
lon_co_1_closest = find_closest_coordinate(lon_coord_1, lon_coords)
lat_co_2_closest = find_closest_coordinate(lat_coord_2, lat_coords)
lon_co_2_closest = find_closest_coordinate(lon_coord_2, lon_coords)

# TODO: Make sure the user knows the difference between given and calculated
# degree (point)

log.info('Calculated coordinates for point 1: ' + str((lat_coord_1, lon_coord_1)))
log.info('Closest coordinates for point 1: ' + str((lat_co_1_closest, lon_co_1_closest)))
log.info('Calculated coordinates for point 2: ' + str((lat_coord_2, lon_coord_2)))
log.info('Closest coordinates for point 2: ' + str((lat_co_2_closest, lon_co_2_closest)))




INFO:notebook:Calculated coordinates for point 1: (287.732532, 301.61934080000003)
INFO:notebook:Closest coordinates for point 1: (288, 302)
INFO:notebook:Calculated coordinates for point 2: (291.073646, 307.1588752)
INFO:notebook:Closest coordinates for point 2: (291, 307)


## Subsetting data

Combining parameter choices above/translation according to OPenDAP guidelines into URL-appendix

In [46]:
'''
"translation" of input to desired URL parameter
Creation of links to relevant datasets (see above) with chosen parameters
add URL parameter to dataset link
dataset links see above
Beispiel-Schema für Wind-Dataset tavg1_2d_slv_Nx (unterteilt in tägliche Datensätze):
Link für Datensatz für 01.01.1980: http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/1980/01/MERRA2_100.tavg1_2d_slv_Nx.19800101.nc4
-> Lädt kompletten Datensatz herunter (alle Parameter, gesamte Welt, 24 Stunden)
-> .html anhängen = manualles subsetting
z.B. time [0:1:23] = jeder Zeitschritt (hier:Stunde) zwischen 0 und 23 Uhr, [0:2:23] = nur jede 2. Stunde etc.
z.B. lat/lon in Schritten angegeben
- latitude (Breitengrad Nord/Süd) 360*0,5°-Schritte
- longitude (Längengrad Ost/West) 575*0,625°-Schritte
'''

def translate_year_to_file_number(year):
    """
    The file names basically consist of a number and a meta data string. 
    The number changes over the year. 1980
    until 1991 it is 100, 1992 until 2000 it is 200, 2001 until 2010 it is 
    300 and from 2011 until now it is 400.
    """
    file_number = ''
    
    if year >= 1980 and year < 1992:
        file_number = '100'
    elif year >= 1992 and year < 2001:
        file_number = '200'
    elif year >= 2001 and year < 2011:
        file_number = '300'
    elif year >= 2011:
        file_number = '400'
    else:
        raise Exception('The specified year is out of range.')
    
    return file_number
    


def generate_url_params(parameter, time_para, lat_para, lon_para):
    '''Creates a string containing all the parameters in query form'''
    parameter = map(lambda x: x + time_para, parameter)
    parameter = map(lambda x: x + lat_para, parameter)
    parameter = map(lambda x: x + lon_para, parameter)
    
    return ','.join(parameter)
    
    

def generate_download_links(download_years, base_url, dataset_name, url_params):
    '''
    Generates the links for the download. 
    download_years: The years you want to download as array. 
    dataset_name: The name of the data set. For example tavg1_2d_slv_Nx
    '''
    urls = []
    for y in download_years: # only for testing
    # build the file_number
        y_str = str(y)
        file_num = translate_year_to_file_number(download_year)
        for m in range(1,13):
            # build the month string: for the month 1 - 9 it starts with a leading 0. zfill solves that problem
            m_str = str(m).zfill(2)
            # monthrange returns the first weekday and the number of days in a month. Also works for leap years.
            _, nr_of_days = monthrange(y, m)
            for d in range(1,nr_of_days+1):
                d_str = str(d).zfill(2)
                file_name = 'MERRA2_' + file_num + '.'+dataset_name+'.' + y_str + m_str + d_str + '.nc4'
                query = base_url  + y_str + '/'+ m_str + '/' + file_name + '.nc4?' + url_params
                urls.append(query)
    return urls

requested_params = ['U2M', 'U10M', 'U50M', 'V2M', 'V10M', 'V50M', 'DISPH']
requested_time = '[0:1:23]'
requested_lat = '[' + str(lat_co_1_closest) + ':1:' + str(lat_co_2_closest) + ']'
requested_lon = '[' + str(lon_co_1_closest) + ':1:' + str(lon_co_2_closest) + ']'



parameter = generate_url_params(requested_params, requested_time,
                                requested_lat, requested_lon)

BASE_URL = 'http://goldsmr4.sci.gsfc.nasa.gov:80/opendap/MERRA2/M2T1NXSLV.5.12.4/'
generated_URL = generate_download_links([download_year], BASE_URL, 
                                        'tavg1_2d_slv_Nx', 
                                        parameter)
            
        
log.debug('Queries generated: ' + str(len(generated_URL)))
log.debug(generated_URL[0])


---
# Downloading data

## Get windspeed of different heights and the displacement height 

In [47]:
# download data (one file per day and dataset) with links to local directory
username = input('Username: ')
password = getpass.getpass('Password:')
download_manager = DownloadManager()
# download_manager.read_credentials_from_yaml('credentials.yaml')
download_manager.set_username_and_password(username, password)
download_manager.download_path = 'download'
download_manager.download_urls = generated_URL
# If you want to see the download progress, check the download folder you specified
%time download_manager.start_download(5)

Username: janurb
Password:········
Wall time: 9min 52s


## Get roughness from different file

In [48]:
roughness_para = generate_url_params(['Z0M'], requested_time, 
                                     requested_lat, requested_lon)
ROUGHNESS_BASE_URL = 'http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/'
roughness_links = generate_download_links([download_year], ROUGHNESS_BASE_URL,
                                          'tavg1_2d_flx_Nx', roughness_para)
            
download_manager.download_path = 'roughness_download'
download_manager.download_urls = roughness_links

# If you want to see the download progress, check the download folder you specified
%time download_manager.start_download(5)

Wall time: 7min 9s


## Get lat and lon dimensions

In [49]:
# The dimensions map the MERRA2 grid coordinates to lat/lon. 
requested_lat_dim = '[' + str(lat_co_1_closest - 1)  + ':1:' + str(lat_co_2_closest - 1) + ']'
requested_lon_dim = '[' + str(lon_co_1_closest - 1)  + ':1:' + str(lon_co_2_closest - 1) + ']'

lat_lon_dimension_para = 'lat' + requested_lat_dim + ',lon' + requested_lon_dim
# Creating the download url.
dimension_url = 'http://goldsmr4.sci.gsfc.nasa.gov:80/opendap/MERRA2/M2T1NXSLV.5.12.4/2014/01/MERRA2_400.tavg1_2d_slv_Nx.20140101.nc4.nc4?'
dimension_url = dimension_url + lat_lon_dimension_para
print(dimension_url)
download_manager.download_path = 'dimension_scale'
download_manager.download_urls = [dimension_url]
# Since the dimension is only one file, we only need one connection. 
%time download_manager.start_download(1)


http://goldsmr4.sci.gsfc.nasa.gov:80/opendap/MERRA2/M2T1NXSLV.5.12.4/2014/01/MERRA2_400.tavg1_2d_slv_Nx.20140101.nc4.nc4?lat[287:1:290],lon[301:1:306]
Wall time: 5.3 s


## Check the precision of the downloaded data

In [50]:
file_path = os.path.join('dimension_scale', DownloadManager.get_filename(
        dimension_url))

with xr.open_dataset(file_path) as ds_dim:
    df_dim = ds_dim.to_dataframe()

lat_array = ds_dim['lat'].data.tolist()
lon_array = ds_dim['lon'].data.tolist()

# The log output helps evaluating the precision of the received data.
log.info('Received lat: ' + str(lat_array))
log.info('Received lon: ' + str(lon_array))
log.info('Requested lat: ' + str((lat_1, lat_2)))
log.info('Requested lon: ' + str((lon_1, lon_2)))


INFO:notebook:Received lat: [53.5, 54.0, 54.5, 55.0]
INFO:notebook:Received lon: [8.125, 8.75, 9.375, 10.0, 10.625, 11.25]
INFO:notebook:Requested lat: (53.366266, 55.036823)
INFO:notebook:Requested lon: (7.887088, 11.349297)


---
# Setting up wind dataframe

In [111]:
def extract_date(data_set):
    """
    Extracts the date from the filename before merging the datasets. 
    """
    # find a match between "." and ".nc4" that does not have "." .
    exp = r'(?<=\.)[^\.]*(?=\.nc4)'
    try:
        if 'HDF5_GLOBAL.Filename' in data_set.attrs:
            f_name = data_set.attrs['HDF5_GLOBAL.Filename']
        elif 'Filename' in data_set.attrs:
            f_name = data_set.attrs['Filename']
        else: 
            raise AttributeError('The attribute name has changed again!')
            
        logging.debug(f_name)
        res = re.search(exp, f_name).group(0)
        y, m, d = res[0:4], res[4:6], res[6:8]
        date_str = ('%s-%s-%s' % (y, m, d))
        data_set = data_set.assign(date=date_str)
        return data_set

    except KeyError:
        # The last dataset is the one all the other sets will be merged into. Therefore, no date can be extracted.
        return data_set
        

file_path = os.path.join('download', '*.nc4')

try:
    with xr.open_mfdataset(file_path, concat_dim='date', preprocess=extract_date) as ds_wind:
        print(ds_wind)
        df_wind = ds_wind.to_dataframe()
        
except xr.MergeError as e:
    print(e)

<xarray.Dataset>
Dimensions:  (date: 365, lat: 4, lon: 6, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4 5
  * date     (date) <U10 '2014-01-01' '2014-01-02' '2014-01-03' '2014-01-04' ...
Data variables:
    U50M     (date, time, lat, lon) float64 -1.312 -1.587 -1.283 -1.846 ...
    U10M     (date, time, lat, lon) float64 -1.18 -1.277 -1.059 -1.574 ...
    V10M     (date, time, lat, lon) float64 7.409 6.227 6.227 6.653 6.888 ...
    V50M     (date, time, lat, lon) float64 9.038 8.616 8.948 8.863 8.753 ...
    U2M      (date, time, lat, lon) float64 -0.9013 -0.885 -0.7202 -1.137 ...
    DISPH    (date, time, lat, lon) float64 0.141 0.2787 0.2898 0.245 0.1877 ...
    V2M      (date, time, lat, lon) float64 5.821 4.311 4.231 4.776 5.176 ...


In [112]:
df_wind.reset_index(inplace=True)

In [113]:
start_date = datetime.strptime(download_start_date, '%Y-%m-%d')

def calculate_datetime(d_frame):
    """
    Calculates the accumulated hour based on the date.
    """
    cur_date = datetime.strptime(d_frame['date'], '%Y-%m-%d')
    hour = int(d_frame['time'])
    delta = abs(cur_date - start_date).days
    date_time_value = (delta * 24) + (hour)
    return date_time_value


df_wind['date_time_hours'] = df_wind.apply(calculate_datetime, axis=1)
df_wind

,date,lat,lon,time,U50M,U10M,V10M,V50M,U2M,DISPH,V2M,date_time_hours
0,2014-01-01,0,0,0,-1.311984,-1.179733,7.409066,9.038284,-0.901322,0.140991,5.820765,0
1,2014-01-01,0,0,1,-0.840221,-0.879409,7.161377,8.794005,-0.666895,0.140930,5.633766,1
2,2014-01-01,0,0,2,-0.993543,-0.919946,6.650461,8.179359,-0.697199,0.140930,5.254070,2
3,2014-01-01,0,0,3,-0.647694,-0.624578,6.702684,8.161971,-0.449956,0.140869,5.308243,3
4,2014-01-01,0,0,4,0.262211,0.091913,6.932301,8.392127,0.115220,0.140869,5.492879,4
5,2014-01-01,0,0,5,1.017545,0.717062,6.883762,8.293762,0.607727,0.140808,5.456675,5
6,2014-01-01,0,0,6,1.427836,1.042843,6.724246,8.050136,0.860338,0.140808,5.329176,6
7,2014-01-01,0,0,7,1.650132,1.256459,6.661009,7.979958,1.029762,0.140747,5.266450,7
8,2014-01-01,0,0,8,2.281217,1.718015,6.780849,8.161178,1.383076,0.140686,5.338470,8
9,2014-01-01,0,0,9,3.251890,2.468855,6.701610,8.092620,1.947835,0.140686,5.253557,9


## Converting the timeformat to ISO 8601 (OPSD Standard)

In [127]:
def converting_timeformat_to_ISO8601(row):
    log.info(row.name)
    log.info(row['date'])
    date = dateutil.parser.parse(row['date'])
    log.info(date)
    hour = int(row['time'])
    log.info(hour)
    date_time = date + timedelta(hours = hour)
#     date_time = pytz.utc.localize(date_time)
    return date_time.isoformat()  # MERRA2 datasets have the UTC time zone.
df_wind['date_utc'] = df.apply(converting_timeformat_to_ISO8601, axis=1)


df_wind.columns.values

INFO:notebook:0
INFO:notebook:2014-01-01T00:00:00Z
INFO:notebook:2014-01-01 00:00:00+00:00
INFO:notebook:0
INFO:notebook:2014-01-01T00:00:00Z
INFO:notebook:2014-01-01 00:00:00+00:00


KeyError: ('time', 'occurred at index 0')

In [105]:
def calculate_windspeed(d_frame, idx_u, idx_v):
    """
    Calculates the windspeed. The returned unit is m/s
    """
    um = int(d_frame[idx_u])
    vm = int(d_frame[idx_v])
    speed = math.sqrt((um ** 2) + (vm ** 2))
    return round(speed, 2)

# partial is used to create a function with pre-set arguments. 
calc_windspeed_2m = partial(calculate_windspeed, idx_u='U2M', idx_v='V2M')
calc_windspeed_10m = partial(calculate_windspeed, idx_u='U10M', idx_v='V10M')
calc_windspeed_50m = partial(calculate_windspeed, idx_u='U50M', idx_v='V50M')

df_wind['v_2m'] = df_wind.apply(calc_windspeed_2m, axis=1)
df_wind['v_10m']= df_wind.apply(calc_windspeed_10m, axis=1)
df_wind['v_50m'] = df_wind.apply(calc_windspeed_50m, axis=1)
df_wind


,date,lat,lon,time,U50M,U10M,V10M,V50M,U2M,DISPH,V2M,date_time_hours,v_2m,v_10m,v_50m
0,2014-01-01,0,0,0,-1.311984,-1.179733,7.409066,9.038284,-0.901322,0.140991,5.820765,0,5.00,7.07,9.06
1,2014-01-01,0,0,1,-0.840221,-0.879409,7.161377,8.794005,-0.666895,0.140930,5.633766,1,5.00,7.00,8.00
2,2014-01-01,0,0,2,-0.993543,-0.919946,6.650461,8.179359,-0.697199,0.140930,5.254070,2,5.00,6.00,8.00
3,2014-01-01,0,0,3,-0.647694,-0.624578,6.702684,8.161971,-0.449956,0.140869,5.308243,3,5.00,6.00,8.00
4,2014-01-01,0,0,4,0.262211,0.091913,6.932301,8.392127,0.115220,0.140869,5.492879,4,5.00,6.00,8.00
5,2014-01-01,0,0,5,1.017545,0.717062,6.883762,8.293762,0.607727,0.140808,5.456675,5,5.00,6.00,8.06
6,2014-01-01,0,0,6,1.427836,1.042843,6.724246,8.050136,0.860338,0.140808,5.329176,6,5.00,6.08,8.06
7,2014-01-01,0,0,7,1.650132,1.256459,6.661009,7.979958,1.029762,0.140747,5.266450,7,5.10,6.08,7.07
8,2014-01-01,0,0,8,2.281217,1.718015,6.780849,8.161178,1.383076,0.140686,5.338470,8,5.10,6.08,8.25
9,2014-01-01,0,0,9,3.251890,2.468855,6.701610,8.092620,1.947835,0.140686,5.253557,9,5.10,6.32,8.54


# Setting up Roughness dataframe

In [55]:
file_path = os.path.join('roughness_download', '*.nc4')
with xr.open_mfdataset(file_path, concat_dim='date', 
                       preprocess=extract_date) as ds_rough:
    df_rough = ds_rough.to_dataframe()

df_rough.reset_index(inplace=True)

df_rough

,date,lat,lon,time,Z0M
0,2014-01-01,0,0,0,0.027990
1,2014-01-01,0,0,1,0.027967
2,2014-01-01,0,0,2,0.027936
3,2014-01-01,0,0,3,0.027936
4,2014-01-01,0,0,4,0.027944
5,2014-01-01,0,0,5,0.027936
6,2014-01-01,0,0,6,0.027921
7,2014-01-01,0,0,7,0.027913
8,2014-01-01,0,0,8,0.027913
9,2014-01-01,0,0,9,0.027906


## Combining the wind and roughness dataframe

In [56]:
df = pd.merge(df_wind, df_rough, on=['date', 'lat', 'lon', 'time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210240 entries, 0 to 210239
Data columns (total 16 columns):
date         210240 non-null object
lat          210240 non-null int64
lon          210240 non-null int64
time         210240 non-null int64
U50M         210240 non-null float64
U10M         210240 non-null float64
V10M         210240 non-null float64
V50M         210240 non-null float64
U2M          210240 non-null float64
DISPH        210240 non-null float64
V2M          210240 non-null float64
date_time    210240 non-null int64
v_2m         210240 non-null float64
v_10m        210240 non-null float64
v_50m        210240 non-null float64
Z0M          210240 non-null float64
dtypes: float64(11), int64(4), object(1)
memory usage: 27.3+ MB


---
# Structure the dataframe, add and remove columns

## Calculating the height with displacement height

In [57]:
# Calculate height for v_2m and v_10m (2 + DISPH or 10 + DISPH)
df['h_v1'] = df.apply((lambda x:int(x['DISPH']) + 2), axis=1)
df['h_v2'] = df.apply((lambda x:int(x['DISPH']) + 10), axis=1)

## Adding needed and removing not needed columns

In [59]:
df['v_100m'] = np.nan
df.drop('DISPH', axis=1, inplace=True)
df.drop(['time'], axis=1, inplace=True)
df.drop(['U2M', 'U10M', 'U50M', 'V2M', 'V10M', 'V50M'], axis=1, inplace=True)
df

,date,lat,lon,date_time,v_2m,v_10m,v_50m,Z0M,h_v1,h_v2,v_100m
0,2014-01-01T00:00:00Z,0,0,0,5.00,7.07,9.06,0.027990,2,10,NaN
1,2014-01-01T00:00:00Z,0,0,1,5.00,7.00,8.00,0.027967,2,10,NaN
2,2014-01-01T00:00:00Z,0,0,2,5.00,6.00,8.00,0.027936,2,10,NaN
3,2014-01-01T00:00:00Z,0,0,3,5.00,6.00,8.00,0.027936,2,10,NaN
4,2014-01-01T00:00:00Z,0,0,4,5.00,6.00,8.00,0.027944,2,10,NaN
5,2014-01-01T00:00:00Z,0,0,5,5.00,6.00,8.06,0.027936,2,10,NaN
6,2014-01-01T00:00:00Z,0,0,6,5.00,6.08,8.06,0.027921,2,10,NaN
7,2014-01-01T00:00:00Z,0,0,7,5.10,6.08,7.07,0.027913,2,10,NaN
8,2014-01-01T00:00:00Z,0,0,8,5.10,6.08,8.25,0.027913,2,10,NaN
9,2014-01-01T00:00:00Z,0,0,9,5.10,6.32,8.54,0.027906,2,10,NaN


## Renaming the columns

In [60]:
df['lat'] = df['lat'].apply(lambda x: lat_array[int(x)])
df['lon'] = df['lon'].apply(lambda x: lon_array[int(x)])

rename_map = {'date_time_hours': 'date/time', 
              'v_2m': 'v1', 
              'v_10m': 'v2', 
              'Z0M': 'z0'
             }

df.rename(columns=rename_map, inplace=True)

# Change order of the columns
columns = ['date/time', 'date', 'lat', 'lon',
        'v1', 'v2', 'v_50m', 'v_100m',
        'h_v1', 'h_v2', 'z0']
df = df[columns]

---
## First look at the final data frame structure and format

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210240 entries, 0 to 210239
Data columns (total 11 columns):
date/time    210240 non-null int64
date         210240 non-null object
lat          210240 non-null float64
lon          210240 non-null float64
v1           210240 non-null float64
v2           210240 non-null float64
v_50m        210240 non-null float64
v_100m       0 non-null float64
h_v1         210240 non-null int64
h_v2         210240 non-null int64
z0           210240 non-null float64
dtypes: float64(7), int64(3), object(1)
memory usage: 19.2+ MB


## Saving dataframe
Save the final dataframe locally

In [62]:
df.to_csv('weather_script_result.csv', index=False)

---
The further processing of the data frame and the compiling of the final data package can be found in [Part3](weather_data_3_processing.ipynb) of this script.

In [63]:
df

,date/time,date,lat,lon,v1,v2,v_50m,v_100m,h_v1,h_v2,z0
0,0,2014-01-01T00:00:00Z,53.5,8.125,5.00,7.07,9.06,NaN,2,10,0.027990
1,1,2014-01-01T00:00:00Z,53.5,8.125,5.00,7.00,8.00,NaN,2,10,0.027967
2,2,2014-01-01T00:00:00Z,53.5,8.125,5.00,6.00,8.00,NaN,2,10,0.027936
3,3,2014-01-01T00:00:00Z,53.5,8.125,5.00,6.00,8.00,NaN,2,10,0.027936
4,4,2014-01-01T00:00:00Z,53.5,8.125,5.00,6.00,8.00,NaN,2,10,0.027944
5,5,2014-01-01T00:00:00Z,53.5,8.125,5.00,6.00,8.06,NaN,2,10,0.027936
6,6,2014-01-01T00:00:00Z,53.5,8.125,5.00,6.08,8.06,NaN,2,10,0.027921
7,7,2014-01-01T00:00:00Z,53.5,8.125,5.10,6.08,7.07,NaN,2,10,0.027913
8,8,2014-01-01T00:00:00Z,53.5,8.125,5.10,6.08,8.25,NaN,2,10,0.027913
9,9,2014-01-01T00:00:00Z,53.5,8.125,5.10,6.32,8.54,NaN,2,10,0.027906
